In [1]:
#from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt

from ortools.sat.python import cp_model

#-----------------------------------EXTRACT THE DATA-----------------------------------

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = 'eua-dataset\\users\\'
U = pd.read_csv(users_path + 'users-test.csv')

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
servers_path = 'eua-dataset\\edge-servers\\'
S = pd.read_csv(servers_path + 'serverstest.csv')

#-----------------------------------VARIABLES-----------------------------------

dij = []  #Distance between server i and user j

D = np.zeros((4), dtype=int) # D = [CPU, disk I/O,.....]

#User u has memory requirements = w (example for 8 users)
w1 = [1, 2, 1, 2]
w2 = [1, 2,	1, 2]
w3 = [5, 7,	6, 6]
w4 = [2, 3,	3, 4]
w5 = [5, 7,	6, 6]
w6 = [2, 3,	3, 4]
w7 = [1, 2, 1, 2]
w8 = [2, 3,	3, 4]
W = [w1, w2, w3, w4, w5, w6, w7, w8]

# Each server has memory capacity c (example for 3 servers)
c1 = [4, 6, 6, 10]
c2 = [10, 17, 15, 16]
c3 = [10, 15, 11, 15]
C = [c1, c2, c3]

#-----------------------------------COVERAGE OF EACH SERVER-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

#radius = np.round(np.random.uniform(0.45,0.75,len(S)),3) #in Km (for each server --> 3 Servers in example)
radius = [0.03, 0.03, 0.03] ####

for i in range(len(S)):
  server_latitude = S.iloc[i]['LATITUDE']
  server_longitude = S.iloc[i]['LONGITUDE']
  dist_temp = []
  for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']

    dist_geo = haversine(server_longitude, server_latitude, user_longitude, user_latitude) #distance between User and Server
    dist_temp.append(np.round(dist_geo,3))

  dij.append(dist_temp)

#-----------------------------------MIP SOLVER-----------------------------------
model = cp_model.CpModel()


#-----------------------------------SOLVER VARIABLES-----------------------------------
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in range(len(U)):
    for i in range(len(S)):
        x[i, j] = model.NewBoolVar(f'c[{i}][{j}]')

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in range(len(S)):
    y[i] = model.NewBoolVar(f'c[{i}]')

#-----------------------------------CONSTRAINTS-----------------------------------

# Capacity constraint
for i in range(len(S)):
    for k in range(len(D)):
        model.Add(
            sum([
                x[i, j] * W[j][k] for j in range(len(U))
            ]) <= C[i][k]*y[i]
        )

#Proximity constraint
for j in range(len(U)):
    for i in range(len(S)): 
        if dij[i][j]>= radius[i]:
            model.Add(x[i,j]==0)

# Each user can be allocated just once
for j in range(len(U)):
    model.Add(
        sum([
            x[i, j] for i in range(len(S))
        ]) <= 1
    )
        
print(model.ModelStats())

satisfaction model '':
#Variables: 27
  - 27 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 12 (#terms: 108)


In [2]:
#-----------------------------------OBJECTIVE FUNCTION-----------------------------------
solver = cp_model.CpSolver()

# Maximize the number of allocated users
objective_max = []
for j in range(len(U)):
    for i in range(len(S)):
        objective_max.append(x[i, j])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_users = solver.ObjectiveValue()

# Hint (speed up solving)
for j in range(len(U)):
    for i in range(len(S)):
        model.AddHint(x[i,j], solver.Value(x[i,j]))

for i in range(len(S)):
    model.AddHint(y[i], solver.Value(y[i]))
        
# constraint prev objective
model.Add(
    sum([
        x[i, j] for i in range(len(S)) for j in range(len(U))
    ]) == round(solver.ObjectiveValue())
)

# Minimize the number of servers used
objective_min = []
for i in range(len(S)):
    objective_min.append(y[i])
model.Minimize(sum(objective_min))

print(model.ModelStats())

optimization model '':
#Variables: 27 (#bools:3 in objective)
  - 27 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 13 (#terms: 132)


In [3]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)

#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    print(f'Objective value: {max_users} users have been allocated to {solver.ObjectiveValue()} servers\n')
    #print(f'Objective value = {solver.Value(x)}\n')
    for j in range(len(U)):
        for i in range(len(S)):
            if int(solver.Value(x[i,j])) == 1:
                print(f'User {j} has been allocated on node {i}')

else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value: 8.0 users have been allocated to 2.0 servers

User 0 has been allocated on node 2
User 1 has been allocated on node 1
User 2 has been allocated on node 2
User 3 has been allocated on node 1
User 4 has been allocated on node 1
User 5 has been allocated on node 1
User 6 has been allocated on node 2
User 7 has been allocated on node 2
